# Práctica: Normalización de Texto para Análisis Multilingüe en Reseñas de Productos
**Contexto:** Eres parte del equipo de NLP de **GlobalReviews**, una empresa que procesa reseñas de usuarios en español, francés e inglés para detectar defectos en productos electrónicos.

## Fase 1: Diagnóstico de Problemas
**Objetivo:** Identificar errores de codificación, acentos inconsistentes y contracciones.

In [30]:
# Celda 1: Cargar librerías iniciales
import pandas as pd
import chardet

# Detectar codificación
with open('resenas_multilingue.csv', 'rb') as f:
    result = chardet.detect(f.read())
print("Codificación detectada:", result['encoding'])


Codificación detectada: Windows-1254


In [31]:
# Crear una versión limpia del CSV con las columnas de texto correctamente citadas

# Leer el archivo con encoding detectado y procesar
with open('resenas_multilingue.csv', 'r', encoding='latin1') as file:
    lines = file.readlines()

# Abrir archivo de salida para escribir la versión limpia
with open('resenas_multilingue_limpio.csv', 'w', encoding='utf-8') as output_file:
    # Escribir encabezado
    header = lines[0].strip().split(',')
    output_file.write(f'"{header[0]}",{",".join(header[1:])}\n')
    
    # Procesar cada línea
    for line in lines[1:]:
        line = line.strip()
        
        # Dividir la línea por comas
        parts = line.split(',')
        
        # Si hay más de 4 partes, significa que hay comas en el texto
        if len(parts) > 4:
            # El texto incluye comas, juntamos todas las partes excepto las últimas 3
            text = ','.join(parts[:-3])
            rest = parts[-3:]  # Los últimos 3 campos
        else:
            # Formato normal (4 partes o menos)
            text = parts[0]
            rest = parts[1:]
        
        # Asegurar que el texto esté entre comillas
        if not (text.startswith('"') and text.endswith('"')):
            text = f'"{text}"'
        
        # Escribir la línea procesada
        output_file.write(f"{text},{','.join(rest)}\n")

print("Archivo CSV limpio creado: resenas_multilingue_limpio.csv")

Archivo CSV limpio creado: resenas_multilingue_limpio.csv


In [32]:
# Celda 2: Cargar el CSV limpio en un DataFrame
df = pd.read_csv('resenas_multilingue_limpio.csv', encoding='utf-8')
print(f"Se cargaron {len(df)} reseñas")
df.head()

Se cargaron 29 reseñas


,texto,idioma,rating,categoria
0,Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...,es,2,electrÃ³nicos
1,J'adore cet ordinateur! Mais le clavier est tr...,fr,4,electrÃ³nicos
2,This product is a disaster... don't buy it! #W...,en,1,electrÃ³nicos
3,"La camisa es bonita, pero se encogiÃÂ³ despuÃ...",es,2,ropa
4,Le service clientÃÂ¨le est nul!!! Je ne recom...,fr,1,servicios


In [33]:
# Celda 3: Buscar ejemplos con texto corrupto
corruptos = df[df['texto'].str.contains('Ã|Â|â|ð|�', na=False)]
corruptos.head()


,texto,idioma,rating,categoria
0,Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...,es,2,electrÃ³nicos
1,J'adore cet ordinateur! Mais le clavier est tr...,fr,4,electrÃ³nicos
2,This product is a disaster... don't buy it! #W...,en,1,electrÃ³nicos
3,"La camisa es bonita, pero se encogiÃÂ³ despuÃ...",es,2,ropa
4,Le service clientÃÂ¨le est nul!!! Je ne recom...,fr,1,servicios


## Fase 2: Normalización Unicode y Codificación
**Objetivo:** Corregir textos corruptos y unificar formatos Unicode.

In [34]:
# Corregir codificación y normalizar Unicode
import unicodedata
def normalize_text(text):
    # Corregir codificación
    text = text.encode('latin1').decode('utf-8')
    # Normalizar Unicode
    text = unicodedata.normalize('NFC', text)
    return text
df['texto_normalizado'] = df['texto'].apply(normalize_text)
df.head()

,texto,idioma,rating,categoria,texto_normalizado
0,Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...,es,2,electrÃ³nicos,Ã‰ste celular es increÃ­ble!!! Pero tarda 5hs ...
1,J'adore cet ordinateur! Mais le clavier est tr...,fr,4,electrÃ³nicos,J'adore cet ordinateur! Mais le clavier est tr...
2,This product is a disaster... don't buy it! #W...,en,1,electrÃ³nicos,This product is a disaster... don't buy it! #W...
3,"La camisa es bonita, pero se encogiÃÂ³ despuÃ...",es,2,ropa,"La camisa es bonita, pero se encogiÃ³ despuÃ©s..."
4,Le service clientÃÂ¨le est nul!!! Je ne recom...,fr,1,servicios,Le service clientÃ¨le est nul!!! Je ne recomma...


## Fase 3: Manejo de Acentos y Contracciones
**Objetivo:** Eliminar acentos opcionales y expandir contracciones coloquiales.

In [37]:
# Manejo de acentos y contracciones
from unidecode import unidecode
import re
def clean_text(text):
    # Eliminar acentos
    text = unidecode(text)
    # Expandir contracciones
    contractions = {"q'huvo": "que hubo", "pq": "porque", " x ": " por "}
    for contraction, full_form in contractions.items():
        text = re.sub(rf'\b{contraction}\b', full_form, text)
    # Normalizar hashtags
    text = re.sub(r'#(\w+)', lambda m: f'#{unidecode(m.group(1).lower())}', text)
    return text
df['texto_limpio'] = df['texto_normalizado'].apply(clean_text)
df.head()

,texto,idioma,rating,categoria,texto_normalizado,texto_limpio
0,Ãâ°ste celular es increÃÂ­ble!!! Pero tarda...,es,2,electrÃ³nicos,Ã‰ste celular es increÃ­ble!!! Pero tarda 5hs ...,A%0ste celular es increAble!!! Pero tarda 5hs ...
1,J'adore cet ordinateur! Mais le clavier est tr...,fr,4,electrÃ³nicos,J'adore cet ordinateur! Mais le clavier est tr...,J'adore cet ordinateur! Mais le clavier est tr...
2,This product is a disaster... don't buy it! #W...,en,1,electrÃ³nicos,This product is a disaster... don't buy it! #W...,This product is a disaster... don't buy it! #w...
3,"La camisa es bonita, pero se encogiÃÂ³ despuÃ...",es,2,ropa,"La camisa es bonita, pero se encogiÃ³ despuÃ©s...","La camisa es bonita, pero se encogiA3 despuA(c..."
4,Le service clientÃÂ¨le est nul!!! Je ne recom...,fr,1,servicios,Le service clientÃ¨le est nul!!! Je ne recomma...,"Le service clientA""le est nul!!! Je ne recomma..."


## Fase 4: Evaluación de Impacto
**Objetivo:** Medir cómo la normalización afecta la calidad del análisis.

In [38]:
# Evaluar impacto de la normalización
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Vectorizar texto
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['texto_limpio'])
y = df['rating']
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Entrenar modelo
model = LogisticRegression()
model.fit(X_train, y_train)
# Evaluar modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 16.67%
